# Unification of Knowledge: Creation of new input

Concatenating the predictors with the responses of the LLM generated using various prompts from the refinement process (e.g., Resp_aware_in = preds + "YES" + "NO" + "YES"). These new inputs were then passed through a ML model for classification.


In [91]:
import pandas as pd
import numpy as np
import time
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix, recall_score, matthews_corrcoef
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB, ComplementNB

In [92]:
data_change = pd.read_csv("../../dat/dips/DIPS_Data_cleaned_change.csv", sep =",", low_memory = False)

In [93]:
data_change

,whi_ges,soz_ges,gke_ges,lzh_ges,ile,woc_gesp,woc_gese,asi_ges,bsq_ges,das_ges,...,ses_kom,whi_change,soz_change,gke_change,lzh_change,asi_change,bsq_change,das_change,scl_change,hpi
0,1.905045,-1.858903,-0.994979,-1.928212,2.441714,0.062486,-0.359417,1.612646,0.764021,0.566822,...,0,-2.301869,2.753129,2.029671,1.900449,-2.518013,-1.431081,-2.600931,-2.639135,0.0
1,0.455324,-0.434443,-0.088340,0.660932,0.041981,0.413949,-1.169769,0.423639,-0.382329,0.941935,...,1,-0.752017,0.204409,0.647389,-0.211204,-0.585676,-0.262880,0.105940,-0.073737,0.0
2,3.838008,-0.701529,-0.315000,-1.280926,0.041981,-0.113246,1.261288,0.291528,-0.286800,0.441785,...,0,-0.752017,0.315223,0.094477,0.316709,1.070613,-0.262880,-0.164747,1.783966,0.0
3,-0.511158,0.990016,0.138320,0.175467,-0.957908,-1.079771,-0.521487,-0.369032,0.477433,0.024994,...,1,-0.235399,-0.017219,0.647389,0.492680,-0.723700,-1.041680,-0.489571,0.147418,0.0
4,-0.027917,-1.947932,0.591639,-1.280926,-0.357975,-0.640441,-0.440452,-1.690150,-1.337621,1.525442,...,0,-0.235399,3.639640,0.923846,2.476355,0.518517,0.321221,-3.034031,-0.162199,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,-0.511158,0.277787,-1.221639,-0.148176,-1.157886,-1.343368,-0.359417,0.555751,-0.018124,0.275069,...,0,0.281218,-0.460475,0.370933,0.844622,-0.723700,0.826225,-1.355770,0.893382,0.0
1225,-0.511158,-0.790558,-0.541660,0.822753,-0.757930,-0.376844,-0.278381,1.084199,0.477433,1.066972,...,0,0.281218,0.758478,-1.011349,-1.794945,0.242469,-0.068179,-1.518183,-0.692971,0.0
1226,-0.511158,-0.523472,-0.315000,0.013646,0.241958,0.238217,0.288865,-1.293815,-1.242092,-0.308439,...,0,0.281218,-0.349661,0.370933,1.372536,1.346661,0.710621,-0.814396,0.191649,0.0
1227,-0.027917,-1.680845,0.364979,-0.471819,0.241958,-0.376844,0.369900,0.159416,0.286375,0.691860,...,0,-0.752017,-0.128033,-0.734893,0.492680,-0.171604,-0.262880,-0.110610,-1.975670,0.0


In [94]:
y_pred_lr = pd.read_csv("../01_ML/y_pred_ML/y_pred_lr.csv", sep = ",")
y_pred_rf = pd.read_csv("../01_ML/y_pred_ML/y_pred_rf.csv", sep = ",")
y_pred_svc = pd.read_csv("../01_ML/y_pred_ML/y_pred_svc.csv", sep = ",")
y_pred_dnn = pd.read_csv("../01_ML/y_pred_ML/y_pred_dnn.csv", sep = ",")
y_pred_knn = pd.read_csv("../01_ML/y_pred_ML/y_pred_knn.csv", sep = ",")
y_pred_nb = pd.read_csv("../01_ML/y_pred_ML/y_pred_nb.csv", sep = ",")
y_pred_xgb = pd.read_csv("../01_ML/y_pred_ML/y_pred_xgb.csv", sep = ",")

In [95]:
y_pred_GPT_simple_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/GPT/y_pred_GPT_4_simple_prompt.csv", sep =",")
y_pred_GPT_class_def_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/GPT/y_pred_GPT_4_class_definitions_prompt.csv", sep =",")
y_pred_GPT_profiled_simple_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/GPT/y_pred_GPT_4_profiled_simple_prompt.csv", sep =",")
y_pred_GPT_few_shot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/GPT/y_pred_GPT_4_few_shot_prompt.csv", sep =",")
y_pred_GPT_vignette_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/GPT/y_pred_GPT_4_vignette_prompt.csv", sep =",")
y_pred_GPT_cot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/GPT/y_pred_GPT_4_cot_prompt.csv", sep =",")

In [96]:
y_pred_GPT_o3_simple_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/GPT/y_pred_GPT_o3_simple_prompt.csv", sep =",")
y_pred_GPT_o3_class_def_prompt = pd.read_csv(
    "../02_LLM/y_pred_LLMs/GPT/y_pred_GPT_o3_class_definitions_prompt.csv", sep =",")
y_pred_GPT_o3_profiled_simple_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/GPT/y_pred_GPT_o3_profiled_simple_prompt.csv", sep =",")
y_pred_GPT_o3_few_shot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/GPT/y_pred_GPT_o3_few_shot_prompt.csv", sep =",")
y_pred_GPT_o3_vignette_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/GPT/y_pred_GPT_o3_vignette_prompt.csv", sep =",")
y_pred_GPT_o3_cot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/GPT/y_pred_GPT_o3_cot_prompt.csv", sep =",")

In [97]:
y_pred_Gemini_simple_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Gemini/y_pred_gemini_simple_prompt.csv", sep =",")
y_pred_Gemini_class_def_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Gemini/y_pred_gemini_class_definitions_prompt.csv", sep =",")
y_pred_Gemini_profiled_simple_prompt = pd.read_csv(
    "../02_LLM/y_pred_LLMs/Gemini/y_pred_gemini_profiled_simple_prompt.csv", sep =",")
y_pred_Gemini_few_shot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Gemini/y_pred_gemini_few_shot_prompt.csv", sep =",")
y_pred_Gemini_vignette_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Gemini/y_pred_gemini_vignette_prompt.csv", sep =",")
y_pred_Gemini_cot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Gemini/y_pred_gemini_cot_prompt.csv", sep =",")

In [98]:
y_pred_Gemma_simple_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Gemma/y_pred_gemma_simple_prompt.csv", sep =",")
y_pred_Gemma_class_def_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Gemma/y_pred_gemma_class_definitions_prompt.csv", sep =",")
y_pred_Gemma_profiled_simple_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Gemma/y_pred_gemma_profiled_simple_prompt.csv", sep =",")
y_pred_Gemma_few_shot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Gemma/y_pred_gemma_few_shot_prompt.csv", sep =",")
y_pred_Gemma_vignette_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Gemma/y_pred_gemma_vignette_prompt.csv", sep =",")
y_pred_Gemma_cot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Gemma/y_pred_gemma_cot_prompt.csv", sep =",")

In [99]:
y_pred_Claude_simple_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Claude/y_pred_claude_simple_prompt.csv", sep =",")
y_pred_Claude_class_def_prompt = pd.read_csv(
    "../02_LLM/y_pred_LLMs/Claude/y_pred_claude_class_definitions_prompt.csv", sep =",")
y_pred_Claude_profiled_simple_prompt = pd.read_csv(
    "../02_LLM/y_pred_LLMs/Claude/y_pred_claude_profiled_simple_prompt.csv", sep =",")
y_pred_Claude_few_shot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Claude/y_pred_claude_few_shot_prompt.csv", sep =",")
y_pred_Claude_vignette_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Claude/y_pred_claude_vignette_prompt.csv", sep =",")
y_pred_Claude_cot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Claude/y_pred_claude_cot_prompt.csv", sep =",")

In [100]:
y_pred_DeepSeek_simple_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/DeepSeek/y_pred_deeps_simple_prompt.csv", sep =",")
y_pred_DeepSeek_class_def_prompt = pd.read_csv(
    "../02_LLM/y_pred_LLMs/DeepSeek/y_pred_deeps_class_definitions_prompt.csv", sep =",")
y_pred_DeepSeek_profiled_simple_prompt = pd.read_csv(
    "../02_LLM/y_pred_LLMs/DeepSeek/y_pred_deeps_profiled_simple_prompt.csv", sep =",")
y_pred_DeepSeek_few_shot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/DeepSeek/y_pred_deeps_few_shot_prompt.csv", sep =",")
y_pred_DeepSeek_vignette_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/DeepSeek/y_pred_deeps_vignette_prompt.csv", sep =",")
y_pred_DeepSeek_cot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/DeepSeek/y_pred_deeps_cot_prompt.csv", sep =",")

In [101]:
y_pred_Grok_simple_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Grok/y_pred_Grok_simple_prompt.csv", sep =",")
y_pred_Grok_class_def_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Grok/y_pred_Grok_class_definitions_prompt.csv", sep =",")
y_pred_Grok_profiled_simple_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Grok/y_pred_Grok_profiled_simple_prompt.csv", sep =",")
y_pred_Grok_few_shot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Grok/y_pred_Grok_few_shot_prompt.csv", sep =",")
y_pred_Grok_vignette_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Grok/y_pred_Grok_vignette_prompt.csv", sep =",")
y_pred_Grok_cot_prompt = pd.read_csv("../02_LLM/y_pred_LLMs/Grok/y_pred_Grok_cot_prompt.csv", sep =",")

In [102]:
X_train_GPT_4_simple_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/GPT/X_train_GPT_4_simple_prompt.csv", sep = ",")
X_train_GPT_4_class_def_prompt = pd.read_csv(
    "New_Input_Prompting/X_train_pred/GPT/X_train_GPT_4_class_definitions_prompt.csv", sep = ",")
X_train_GPT_4_profiled_simple_prompt = pd.read_csv(
    "New_Input_Prompting/X_train_pred/GPT/X_train_GPT_4_profiled_simple_prompt.csv", sep = ",")
X_train_GPT_4_few_shot_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/GPT/X_train_GPT_4_few_shot_prompt.csv", sep = ",")
X_train_GPT_4_vignette_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/GPT/X_train_GPT_4_vignette_prompt.csv", sep = ",")
X_train_GPT_4_cot_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/GPT/X_train_GPT_4_cot_prompt.csv", sep = ",")

In [103]:
X_train_GPT_o3_simple_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/GPT/X_train_GPT_o3_simple_prompt.csv", sep = ",")
X_train_GPT_o3_class_def_prompt = pd.read_csv(
    "New_Input_Prompting/X_train_pred/GPT/X_train_GPT_o3_class_definitions_prompt.csv", sep = ",")
X_train_GPT_o3_profiled_simple_prompt = pd.read_csv(
    "New_Input_Prompting/X_train_pred/GPT/X_train_GPT_o3_profiled_simple_prompt.csv", sep = ",")
X_train_GPT_o3_few_shot_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/GPT/X_train_GPT_o3_few_shot_prompt.csv", sep = ",")
X_train_GPT_o3_vignette_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/GPT/X_train_GPT_o3_vignette_prompt.csv", sep = ",")
X_train_GPT_o3_cot_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/GPT/X_train_GPT_o3_cot_prompt.csv", sep = ",")

In [104]:
X_train_Gemma_simple_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/Gemma/X_train_Gemma_simple_prompt.csv", sep = ",")
X_train_Gemma_class_def_prompt = pd.read_csv(
    "New_Input_Prompting/X_train_pred/Gemma/X_train_Gemma_class_definitions_prompt.csv", sep = ",")
X_train_Gemma_profiled_simple_prompt = pd.read_csv(
    "New_Input_Prompting/X_train_pred/Gemma/X_train_Gemma_profiled_simple_prompt.csv", sep = ",")
X_train_Gemma_few_shot_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/Gemma/X_train_Gemma_few_shot_prompt.csv", sep = ",")
X_train_Gemma_vignette_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/Gemma/X_train_Gemma_vignette_prompt.csv", sep = ",")
X_train_Gemma_cot_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/Gemma/X_train_Gemma_cot_prompt.csv", sep = ",")

In [105]:
X_train_Claude_simple_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/Claude/X_train_Claude_simple_prompt.csv", sep = ",")
X_train_Claude_class_def_prompt = pd.read_csv(
    "New_Input_Prompting/X_train_pred/Claude/X_train_Claude_class_definitions_prompt.csv", sep = ",")
X_train_Claude_profiled_simple_prompt = pd.read_csv(
    "New_Input_Prompting/X_train_pred/Claude/X_train_Claude_profiled_simple_prompt.csv", sep = ",")
X_train_Claude_few_shot_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/Claude/X_train_Claude_few_shot_prompt.csv", sep = ",")
X_train_Claude_vignette_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/Claude/X_train_Claude_vignette_prompt.csv", sep = ",")
X_train_Claude_cot_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/Claude/X_train_Claude_cot_prompt.csv", sep = ",")

In [106]:
X_train_DeepSeek_simple_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/DeepSeek/X_train_deeps_simple_prompt.csv", sep = ",")
X_train_DeepSeek_class_def_prompt = pd.read_csv(
    "New_Input_Prompting/X_train_pred/DeepSeek/X_train_deeps_class_definitions_prompt.csv", sep = ",")
X_train_DeepSeek_profiled_simple_prompt = pd.read_csv(
    "New_Input_Prompting/X_train_pred/DeepSeek/X_train_deeps_profiled_simple_prompt.csv", sep = ",")
X_train_DeepSeek_few_shot_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/DeepSeek/X_train_deeps_few_shot_prompt.csv", sep = ",")
X_train_DeepSeek_vignette_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/DeepSeek/X_train_deeps_vignette_prompt.csv", sep = ",")
X_train_DeepSeek_cot_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/DeepSeek/X_train_deeps_cot_prompt.csv", sep = ",")

In [107]:
X_train_Grok_simple_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/Grok/X_train_grok_simple_prompt.csv", sep = ",")
X_train_Grok_class_def_prompt = pd.read_csv(
    "New_Input_Prompting/X_train_pred/Grok/X_train_grok_class_definitions_prompt.csv", sep = ",")
X_train_Grok_profiled_simple_prompt = pd.read_csv(
    "New_Input_Prompting/X_train_pred/Grok/X_train_grok_profiled_simple_prompt.csv", sep = ",")
X_train_Grok_few_shot_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/Grok/X_train_grok_few_shot_prompt.csv", sep = ",")
X_train_Grok_vignette_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/Grok/X_train_grok_vignette_prompt.csv", sep = ",")
X_train_Grok_cot_prompt = pd.read_csv("New_Input_Prompting/X_train_pred/Grok/X_train_grok_cot_prompt.csv", sep = ",")

## 1 Naive Bayes Model

In [114]:
def sklearn_naive_bayes(x_test_nb, x_train_nb, y_train_nb, y_test_nb):

    # 1: GRID SEARCH
    gnb_model = GaussianNB()
    param_grid = {
        'var_smoothing': np.logspace(0, -10, 50)
    }
    grid_search_gnb = GridSearchCV(
        estimator = gnb_model,
        param_grid = param_grid,
        cv = 10
    )
    grid_search_gnb.fit(x_train_nb, y_train_nb)
    best_model_gnb = grid_search_gnb.best_estimator_

    bnb_model = BernoulliNB()
    param_grid_bnb = {
        'alpha': np.logspace(0, -10, 50)
    }
    grid_search_bnb = GridSearchCV(
        estimator = bnb_model,
        param_grid = param_grid_bnb,
        cv = 10
    )
    grid_search_bnb.fit(x_train_nb, y_train_nb)
    best_model_bnb = grid_search_bnb.best_estimator_

    # mnb = MultinomialNB()
    # param_grid_mnb = {
    #     'alpha': np.logspace(0, -10, 50)
    # }
    # grid_search_mnb = GridSearchCV(
    #     estimator = mnb,
    #     param_grid = param_grid_mnb,
    #     cv = 10
    # )
    # grid_search_mnb.fit(x_train_nb, y_train_nb)
    # best_model_mnb = grid_search_mnb.best_estimator_

    # cnb = ComplementNB()
    # param_grid_cnb = {
    #     'alpha': np.logspace(0, -10, 50)
    # }
    # grid_search_cnb = GridSearchCV(
    #     estimator = cnb,
    #     param_grid = param_grid_cnb,
    #     cv = 10
    # )
    # grid_search_cnb.fit(x_train_nb, y_train_nb)
    # best_model_cnb = grid_search_cnb.best_estimator_

    # Select the best model based on the best score
    best_model = max(
        [best_model_gnb, best_model_bnb],
        key = lambda model: model.score(x_train_nb, y_train_nb)
    )
    if best_model == best_model_gnb:
        print("Best model: GaussianNB")
        best_model = best_model_gnb
        grid_search = grid_search_gnb
    elif best_model == best_model_bnb:
        print("Best model: BernoulliNB")
        best_model = best_model_bnb
        grid_search = grid_search_bnb
    # elif best_model == best_model_mnb:
    #     print("Best model: MultinomialNB")
    #     best_model = best_model_mnb
    #     grid_search = grid_search_mnb
    # elif best_model == best_model_cnb:
    #     print("Best model: ComplementNB")
    #     best_model = best_model_cnb
    #     grid_search = grid_search_cnb

    print("Best parameters: ", grid_search.best_params_)
    print("Best score: ", grid_search.best_score_)

    # 2: FITTING THE MODEL
    model = GaussianNB(
        var_smoothing = best_model.var_smoothing
    )
    model.fit(x_train_nb, y_train_nb)

    # 3: ESTIMATING WEIGHTS
    weights_nb = model.theta_
    features_nb = model.feature_names_in_

    # 4: COMPUTE TEST SET PREDICTIONS
    y_pred_nb = model.predict(x_test_nb)
    y_pred_proba_nb = model.predict_proba(x_test_nb)

    # 5: COMPUTE METRICS
    accuracy_nb = model.score(x_test_nb, y_test_nb)
    macro_f1_nb = recall_score(y_test_nb, y_pred_nb, average = "macro")
    micro_f1_nb = recall_score(y_test_nb, y_pred_nb, average = "micro")
    mcc_nb = matthews_corrcoef(y_test_nb, y_pred_nb)

    cm_nb = confusion_matrix(y_test_nb, y_pred_nb)
    precision_nb = cm_nb[1][1] / (cm_nb[1][1] + cm_nb[0][1])
    recall_nb = cm_nb[1][1] / (cm_nb[1][1] + cm_nb[1][0])

    # store metrics in a dictionary
    metrics_nb = {
        "accuracy": round(accuracy_nb, 4),
        "macro_f1": round(macro_f1_nb, 4),
        "micro_f1": round(micro_f1_nb, 4),
        "mcc": round(mcc_nb, 4),
        "precision": round(precision_nb, 4),
        "recall": round(recall_nb, 4),
        "confusion_matrix": cm_nb
    }

    return weights_nb, y_pred_nb, features_nb, metrics_nb

In [115]:
# Predictors
X = data_change
X = X.drop(["hpi"], axis = 1)

# Target
y = data_change["hpi"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

print("NB \n",
      "X_train shape: ", X_train.shape, round(X_train.shape[0]/len(X), 2), "\n",
      "X_test shape: ", X_test.shape, round(X_test.shape[0]/len(X), 2),  "\n",
      "y_train shape: ", y_train.shape, round(y_train.shape[0]/len(y), 2), "\n",
      "y_test shape: ", y_test.shape, round(y_test.shape[0]/len(y), 2), "\n")

NB 
 X_train shape:  (983, 22) 0.8 
 X_test shape:  (246, 22) 0.2 
 y_train shape:  (983,) 0.8 
 y_test shape:  (246,) 0.2 



In [116]:
# concatenate the predictors with the responses of the LLM generated using various prompts
X_train = X_train.reset_index(drop = True)

X_train_new_input = pd.concat([
    X_train,

    X_train_GPT_4_simple_prompt["y_pred"],
    X_train_GPT_4_class_def_prompt["y_pred"],
    X_train_GPT_4_profiled_simple_prompt["y_pred"],
    X_train_GPT_4_few_shot_prompt["y_pred"],
    X_train_GPT_4_vignette_prompt["y_pred"],
    X_train_GPT_4_cot_prompt["y_pred"],

    X_train_GPT_o3_simple_prompt["y_pred"],
    X_train_GPT_o3_class_def_prompt["y_pred"],
    X_train_GPT_o3_profiled_simple_prompt["y_pred"],
    X_train_GPT_o3_few_shot_prompt["y_pred"],
    X_train_GPT_o3_vignette_prompt["y_pred"],
    X_train_GPT_o3_cot_prompt["y_pred"],

    X_train_Gemma_simple_prompt["y_pred"],
    X_train_Gemma_class_def_prompt["y_pred"],
    X_train_Gemma_profiled_simple_prompt["y_pred"],
    X_train_Gemma_few_shot_prompt["y_pred"],
    X_train_Gemma_vignette_prompt["y_pred"],
    X_train_Gemma_cot_prompt["y_pred"],

    X_train_Claude_simple_prompt["y_pred"],
    X_train_Claude_class_def_prompt["y_pred"],
    X_train_Claude_profiled_simple_prompt["y_pred"],
    X_train_Claude_few_shot_prompt["y_pred"],
    X_train_Claude_vignette_prompt["y_pred"],
    X_train_Claude_cot_prompt["y_pred"],

    X_train_DeepSeek_simple_prompt["y_pred"],
    X_train_DeepSeek_class_def_prompt["y_pred"],
    X_train_DeepSeek_profiled_simple_prompt["y_pred"],
    X_train_DeepSeek_few_shot_prompt["y_pred"],
    X_train_DeepSeek_vignette_prompt["y_pred"],
    X_train_DeepSeek_cot_prompt["y_pred"],

    X_train_Grok_simple_prompt["y_pred"],
    X_train_Grok_class_def_prompt["y_pred"],
    X_train_Grok_profiled_simple_prompt["y_pred"],
    X_train_Grok_few_shot_prompt["y_pred"],
    X_train_Grok_vignette_prompt["y_pred"],
    X_train_Grok_cot_prompt["y_pred"]
], axis = 1)

# rename columns
X_train_new_input.columns = [
    *X_train.columns,

    "y_pred_GPT_4_simple",
    "y_pred_GPT_4_class_def",
    "y_pred_GPT_4_profiled",
    "y_pred_GPT_4_few_shot",
    "y_pred_GPT_4_vignette",
    "y_pred_GPT_4_cot",

    "y_pred_GPT_o3_simple",
    "y_pred_GPT_o3_class_def",
    "y_pred_GPT_o3_profiled",
    "y_pred_GPT_o3_few_shot",
    "y_pred_GPT_o3_vignette",
    "y_pred_GPT_o3_cot",

    "y_pred_Gemma_simple",
    "y_pred_Gemma_class_def",
    "y_pred_Gemma_profiled",
    "y_pred_Gemma_few_shot",
    "y_pred_Gemma_vignette",
    "y_pred_Gemma_cot",

    "y_pred_Claude_simple",
    "y_pred_Claude_class_def",
    "y_pred_Claude_profiled",
    "y_pred_Claude_few_shot",
    "y_pred_Claude_vignette",
    "y_pred_Claude_cot",

    "y_pred_DeepSeek_simple",
    "y_pred_DeepSeek_class_def",
    "y_pred_DeepSeek_profiled",
    "y_pred_DeepSeek_few_shot",
    "y_pred_DeepSeek_vignette",
    "y_pred_DeepSeek_cot",

    "y_pred_Grok_simple",
    "y_pred_Grok_class_def",
    "y_pred_Grok_profiled",
    "y_pred_Grok_few_shot",
    "y_pred_Grok_vignette",
    "y_pred_Grok_cot",
]

X_train_new_input

,whi_ges,soz_ges,gke_ges,lzh_ges,ile,woc_gesp,woc_gese,asi_ges,bsq_ges,das_ges,...,y_pred_DeepSeek_profiled,y_pred_DeepSeek_few_shot,y_pred_DeepSeek_vignette,y_pred_DeepSeek_cot,y_pred_Grok_simple,y_pred_Grok_class_def,y_pred_Grok_profiled,y_pred_Grok_few_shot,y_pred_Grok_vignette,y_pred_Grok_cot
0,-0.027917,0.188758,-0.768319,-4.988109,0.041981,-0.113246,-0.035276,0.027304,-1.051033,0.108352,...,1,0.0,1,0,1,1,1,1,0,1.0
1,-0.994398,-1.324730,-0.088340,0.013646,-0.357975,-0.201112,1.099218,-0.897479,-0.668917,-0.475156,...,0,0.0,0,0,0,0,0,0,0,0.0
2,-0.994398,0.722930,1.498278,0.660932,-1.157886,0.589681,-1.818051,-0.897479,-0.859975,-1.933925,...,0,1.0,0,1,1,1,1,1,0,0.0
3,-0.511158,-0.790558,-0.768319,0.660932,0.441936,-0.991905,2.071641,-0.104808,-0.000212,0.316748,...,0,1.0,0,0,1,1,1,1,0,0.0
4,-0.027917,0.990016,1.724938,-0.795462,-1.157886,0.853278,-0.359417,-0.236920,0.095317,-1.016984,...,0,0.0,0,1,1,1,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,-0.511158,0.010700,-0.541660,-0.148176,0.641914,-0.288978,-0.116311,1.084199,0.095317,1.400405,...,1,1.0,1,0,1,1,1,1,1,0.0
979,-0.511158,1.079045,0.138320,1.308218,-1.557841,-1.255502,-1.574945,-0.897479,-1.146563,-0.350118,...,0,0.0,1,1,1,0,1,0,0,0.0
980,-0.994398,0.455844,-0.315000,-0.148176,-0.957908,-0.464710,1.342323,-0.236920,-0.859975,0.108352,...,0,0.0,0,0,1,0,0,0,0,0.0
981,-0.511158,0.990016,-1.221639,0.337289,-1.157886,-0.201112,1.909570,0.423639,-0.573388,0.525143,...,0,0.0,0,0,1,0,0,0,0,0.0


In [117]:
X_test = X_test.reset_index(drop = True)

X_test_new_input = pd.concat([
    X_test,

    y_pred_GPT_simple_prompt["y_pred"],
    y_pred_GPT_class_def_prompt["y_pred"],
    y_pred_GPT_profiled_simple_prompt["y_pred"],
    y_pred_GPT_few_shot_prompt["y_pred"],
    y_pred_GPT_vignette_prompt["y_pred"],
    y_pred_GPT_cot_prompt["y_pred"],

    y_pred_GPT_o3_simple_prompt["y_pred"],
    y_pred_GPT_o3_class_def_prompt["y_pred"],
    y_pred_GPT_o3_profiled_simple_prompt["y_pred"],
    y_pred_GPT_o3_few_shot_prompt["y_pred"],
    y_pred_GPT_o3_vignette_prompt["y_pred"],
    y_pred_GPT_o3_cot_prompt["y_pred"],

    y_pred_Gemma_simple_prompt["y_pred"],
    y_pred_Gemma_class_def_prompt["y_pred"],
    y_pred_Gemma_profiled_simple_prompt["y_pred"],
    y_pred_Gemma_few_shot_prompt["y_pred"],
    y_pred_Gemma_vignette_prompt["y_pred"],
    y_pred_Gemma_cot_prompt["y_pred"],

    y_pred_Claude_simple_prompt["y_pred"],
    y_pred_Claude_class_def_prompt["y_pred"],
    y_pred_Claude_profiled_simple_prompt["y_pred"],
    y_pred_Claude_few_shot_prompt["y_pred"],
    y_pred_Claude_vignette_prompt["y_pred"],
    y_pred_Claude_cot_prompt["y_pred"],

    y_pred_DeepSeek_simple_prompt["y_pred"],
    y_pred_DeepSeek_class_def_prompt["y_pred"],
    y_pred_DeepSeek_profiled_simple_prompt["y_pred"],
    y_pred_DeepSeek_few_shot_prompt["y_pred"],
    y_pred_DeepSeek_vignette_prompt["y_pred"],
    y_pred_DeepSeek_cot_prompt["y_pred"],

    y_pred_Grok_simple_prompt["y_pred"],
    y_pred_Grok_class_def_prompt["y_pred"],
    y_pred_Grok_profiled_simple_prompt["y_pred"],
    y_pred_Grok_few_shot_prompt["y_pred"],
    y_pred_Grok_vignette_prompt["y_pred"],
    y_pred_Grok_cot_prompt["y_pred"]
], axis = 1)

X_test_new_input.columns = [
    *X_test.columns,

    "y_pred_GPT_4_simple",
    "y_pred_GPT_4_class_def",
    "y_pred_GPT_4_profiled",
    "y_pred_GPT_4_few_shot",
    "y_pred_GPT_4_vignette",
    "y_pred_GPT_4_cot",

    "y_pred_GPT_o3_simple",
    "y_pred_GPT_o3_class_def",
    "y_pred_GPT_o3_profiled",
    "y_pred_GPT_o3_few_shot",
    "y_pred_GPT_o3_vignette",
    "y_pred_GPT_o3_cot",

    "y_pred_Gemma_simple",
    "y_pred_Gemma_class_def",
    "y_pred_Gemma_profiled",
    "y_pred_Gemma_few_shot",
    "y_pred_Gemma_vignette",
    "y_pred_Gemma_cot",

    "y_pred_Claude_simple",
    "y_pred_Claude_class_def",
    "y_pred_Claude_profiled",
    "y_pred_Claude_few_shot",
    "y_pred_Claude_vignette",
    "y_pred_Claude_cot",

    "y_pred_DeepSeek_simple",
    "y_pred_DeepSeek_class_def",
    "y_pred_DeepSeek_profiled",
    "y_pred_DeepSeek_few_shot",
    "y_pred_DeepSeek_vignette",
    "y_pred_DeepSeek_cot",

    "y_pred_Grok_simple",
    "y_pred_Grok_class_def",
    "y_pred_Grok_profiled",
    "y_pred_Grok_few_shot",
    "y_pred_Grok_vignette",
    "y_pred_Grok_cot",
]

X_test_new_input

,whi_ges,soz_ges,gke_ges,lzh_ges,ile,woc_gesp,woc_gese,asi_ges,bsq_ges,das_ges,...,y_pred_DeepSeek_profiled,y_pred_DeepSeek_few_shot,y_pred_DeepSeek_vignette,y_pred_DeepSeek_cot,y_pred_Grok_simple,y_pred_Grok_class_def,y_pred_Grok_profiled,y_pred_Grok_few_shot,y_pred_Grok_vignette,y_pred_Grok_cot
0,-0.027917,0.142124,0.364979,0.337289,0.441936,1.731937,0.207830,0.159416,0.286375,0.275069,...,0,0,0,0.0,0,0,0,1,0,0.0
1,-0.027917,1.079045,-2.354937,-0.471819,0.041981,0.853278,-0.845628,0.555751,1.241667,-0.225081,...,1,1,0,0.0,1,1,1,1,1,0.0
2,-0.994398,0.811959,2.404917,0.984575,-0.957908,2.698461,-0.764593,-0.369032,0.286375,-1.308738,...,0,0,0,0.0,0,0,0,0,0,0.0
3,-0.027917,0.900988,0.364979,1.631861,-0.957908,0.150351,-0.278381,-1.161703,-1.242092,-1.100342,...,1,0,1,1.0,1,1,1,1,1,1.0
4,-0.511158,0.811959,0.591639,1.470039,-0.957908,-0.288978,-0.602522,-1.029591,-0.764446,-1.016984,...,0,0,0,0.0,1,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,-0.511158,0.722930,0.818299,0.660932,0.841892,-0.376844,-0.926663,-0.236920,-0.382329,-1.683850,...,0,1,1,1.0,1,1,1,1,1,1.0
242,-0.511158,-0.167357,0.364979,0.337289,-1.157886,-1.343368,-0.440452,-0.104808,-0.286800,0.900255,...,0,0,0,0.0,1,0,0,0,0,0.0
243,0.455324,0.277787,-0.541660,-0.309997,-0.957908,1.995534,2.476817,0.687863,-0.955504,-0.641872,...,0,0,1,0.0,1,1,1,0,1,1.0
244,-0.994398,-0.434443,0.138320,0.822753,0.041981,-1.255502,-0.035276,0.555751,-0.095742,1.650480,...,0,0,0,0.0,0,0,0,1,0,0.0


In [118]:
X_train_new_input.isna().sum()

whi_ges                      0
soz_ges                      0
gke_ges                      0
lzh_ges                      0
ile                          0
woc_gesp                     0
woc_gese                     0
asi_ges                      0
bsq_ges                      0
das_ges                      0
scl_gsi                      0
bild                         0
bmi_kat                      0
ses_kom                      0
whi_change                   0
soz_change                   0
gke_change                   0
lzh_change                   0
asi_change                   0
bsq_change                   0
das_change                   0
scl_change                   0
y_pred_GPT_4_simple          0
y_pred_GPT_4_class_def       0
y_pred_GPT_4_profiled        0
y_pred_GPT_4_few_shot        0
y_pred_GPT_4_vignette        0
y_pred_GPT_4_cot             0
y_pred_GPT_o3_simple         0
y_pred_GPT_o3_class_def      0
y_pred_GPT_o3_profiled       0
y_pred_GPT_o3_few_shot       0
y_pred_G

In [119]:
start = time.time()

weights_naive, y_pred_naive, features_naive, metrics_naive = sklearn_naive_bayes(X_test_new_input, X_train_new_input, y_train, y_test)

end = time.time()
print("Execution time: ", {end - start}, "seconds")
time_nb = end - start
time_nb_df = pd.DataFrame({"time": [time_nb]})
time_nb_df.to_csv("y_pred_unification/new_input/time_new_input_nb_X_train.csv", sep = ",", index = False)

# save weights_ML and y_pred_ML
# weights_naive = pd.DataFrame([weights_naive], columns = features_naive)
# weights_naive.to_csv("../exp/weights_ML/weights_nb.csv", sep = ",", index = False)

y_pred_naive = pd.DataFrame(y_pred_naive, columns = ["y_pred"])
y_pred_naive.to_csv("y_pred_unification/new_input/y_pred_new_input_nb_X_train.csv", sep = ",", index = False)

Best model: GaussianNB
Best parameters:  {'var_smoothing': np.float64(1.0)}
Best score:  0.7303854875283446
Execution time:  {10.468024253845215} seconds


In [120]:
metrics_naive

{'accuracy': 0.7398,
 'macro_f1': 0.5937,
 'micro_f1': 0.7398,
 'mcc': np.float64(0.2537),
 'precision': np.float64(0.5806),
 'recall': np.float64(0.2609),
 'confusion_matrix': array([[164,  13],
        [ 51,  18]])}